## Task decomposition in prompts

This notebook explores task decomposition in prompt engineering, a technique that breaks down complex problems into smaller, more manageable subtasks. This approach is key to effectively leveraging LLMs for multi-step reasoning tasks. By decomposing a problem into simpler parts, we improve the model's ability to produce more reliable, accurate, and interpretable results.

While AI language models are increasingly capable of handling complex tasks, they perform best when provided with clear, structured instructions. Task decomposition simplifies complex problems, allowing the model to focus on one element at a time, leading to better outcomes. This not only enhances the model's performance but also improves the interpretability and reliability of the results.

In this notebook, we demonstrate how to break down a financial analysis task into individual subtasks. We will guide the model step by step, resulting in a comprehensive solution.

In [1]:
import os
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from dotenv import load_dotenv

# Load environment variables
load_dotenv()

# Set up OpenAI API key
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

### Initialize the language model
We instantiate a lightweight GPT model from OpenAI using LangChain.

In [2]:
# Initialize the language model
llm = ChatOpenAI(model="gpt-4o-mini-2024-07-18")

We will also define a helper function to run the prompts:

In [3]:
# Define a helper function to run the prompts
def run_prompt(prompt, **kwargs):
    """Helper function to run a prompt through the language model.

    Args:
        prompt (str): The prompt template string.
        **kwargs: Keyword arguments to fill the prompt template.

    Returns:
        str: The model's response.
    """
    # Create a PromptTemplate object using the provided prompt string
    prompt_template = PromptTemplate(template=prompt, input_variables=list(kwargs.keys()))

    # Combine the prompt template with the llm to create a processing chain
    chain = prompt_template | llm

    # Invoke the chain with the provided keyword arguments (which fill in the placeholders in the prompt)
    return chain.invoke(kwargs).content

`run_prompt` is designed to take a prompt (a template with placeholders) and dynamic keyword arguments (`kwargs`) to fill in those placeholders.
- `PromptTemplate` object: This object takes the raw `prompt` string and a list of keys (`kwargs.keys()`) to identify the placeholders in the prompt. It essentially prepares the template by marking where values need to be inserted.
- Chaining `PromptTemplate` and LLM: The prompt template is combined with the language model (`llm`) using the `|` operator. This creates a processing "chain" that will use the prompt template to format the input and then pass it through the model to get a response.
- Invoke and return: `.invoke(kwargs)` runs the chain, passing in the dynamic values as keyword arguments. The model generates a response based on these values, and the function returns just the `content` of the response (i.e., the model's generated text).


## Breaking down complex tasks
In this section, we will break down a complex task into smaller, more manageable subtasks using the example of analyzing a company's financial health. This demonstrates the importance of task decomposition in prompt engineering, where complex problems are split into discrete steps that the AI model can handle more effectively.

The task at hand is to analyze a company's financial health based on specific financial data. However, analyzing financial health involves several components, such as profitability, liquidity, and cash flow, which are better addressed individually. Instead of giving the model one complex prompt to perform all analyses at once, we first decompose the task into multiple smaller tasks, allowing the model to focus on one aspect of the analysis at a time.

In [4]:
# Define the complex task, which is to analyze the financial health of a company
complex_task = """
Analyze the financial health of a company based on the following data:
- Revenue: $10 million
- Net Income: $2 million
- Total Assets: $15 million
- Total Liabilities: $7 million
- Cash Flow from Operations: $3 million
"""

# Define the prompt template to decompose the complex task into subtasks
decomposition_prompt = """
Break down the task of analyzing a company's financial health into 3 subtasks. For each subtask, provide a brief description of what it should accomplish.

Task: {task}

Subtasks:
1.
"""

# Call the function to invoke the decomposition process using the decomposition prompt
subtasks = run_prompt(decomposition_prompt, task=complex_task)
print(subtasks)

### Subtask 1: Assess Profitability
**Description:** Evaluate the company's ability to generate profit relative to its revenue and assets. This involves calculating key profitability ratios such as the net profit margin (Net Income / Revenue) and return on assets (Net Income / Total Assets). The objective is to determine how effectively the company converts sales into profit and how efficiently it utilizes its assets to generate earnings.

---

### Subtask 2: Analyze Liquidity and Solvency
**Description:** Examine the company's ability to meet its short-term and long-term obligations. This includes calculating liquidity ratios like the current ratio (Current Assets / Current Liabilities) and solvency ratios such as the debt-to-equity ratio (Total Liabilities / Total Assets). The goal is to assess the financial stability of the company and its capacity to cover debts as they come due.

---

### Subtask 3: Evaluate Cash Flow Health
**Description:** Investigate the company's cash flow sit

1. Define the complex task (`complex_task`): This block contains the initial task, which is to analyze a company's financial health based on provided data. The task includes essential financial figures like revenue, net income, assets, liabilities, and cash flow. This represents the "big picture" problem that needs to be broken down into smaller, more manageable subtasks.
2. Decomposition prompt (`decomposition_prompt`): The decomposition prompt is crafted to instruct the model to split the given task into 3 distinct subtasks. The prompt includes a placeholder `{task}`, which will be replaced with the `complex_task` content when the prompt is executed. The goal of this prompt is to guide the model to produce three clear and actionable subtasks that focus on different aspects of the financial health analysis.
3. Calling the `run_prompt` function: The `run_prompt` function is invoked, passing in the `decomposition_prompt` template along with the `complex_task`.
   - The function takes the task (financial health analysis) and feeds it into the decomposition prompt.
   - The `run_prompt` function processes the template, replaces the placeholder `{task}` with the task content, and invokes the model to generate the decomposition.

This technique ensures that large and complex tasks are handled more efficiently by focusing the model's attention on one aspect of the problem at a time. It also makes the model's reasoning process more interpretable and reliable, especially in domains like financial analysis where precision is important.


## Chaining subtasks in prompts
Now that we have decomposed the complex task into smaller, more manageable subtasks, we will chain them together. Chaining allows us to guide the language model through a series of steps, where the output of one subtask feeds into the next. This technique is essential when solving multi-step problems, as it enables us to break the overall problem down into smaller parts and solve them sequentially.

### Defining separate prompts for subtasks
In this section, we will create individual prompts for each of the three subtasks we identified earlier: profitability analysis, liquidity analysis, and cash flow analysis. Each subtask will be analyzed separately, and the results will be obtained step by step.
1. Profitability analysis: The first subtask involves analyzing the company's profitability.
2. Liquidity analysis: The second subtask focuses on analyzing the company's liquidity, which is typically evaluated using its total assets and total liabilities.
3. Cash flow analysis: The third subtask involves analyzing the company’s cash flow from operations, which indicates the company’s ability to generate cash from its core operations.

Each of these subtasks will be handled by a separate function, and we will invoke them sequentially. The model will process each subtask independently but will be as part of a larger task.

In [5]:
# Function to analyze the company's profitability
def analyze_profitability(revenue, net_income):
    """Analyze the company's profitability.

    Args:
        revenue (float): Company's revenue.
        net_income (float): Company's net income.

    Returns:
        str: Analysis of the company's profitability.
    """
    # Define the prompt to analyze profitability based on the provided data
    prompt = """
    Analyze the company's profitability based on the following data:
    - Revenue: ${revenue} million
    - Net Income: ${net_income} million

    Calculate the profit margin and provide a brief analysis of the company's profitability.
    """
    # Call the function to get the model's response for profitability analysis
    return run_prompt(prompt, revenue=revenue, net_income=net_income)

# Function to analyze the company's liquidity
def analyze_liquidity(total_assets, total_liabilities):
    """Analyze the company's liquidity.

    Args:
        total_assets (float): Company's total assets.
        total_liabilities (float): Company's total liabilities.

    Returns:
        str: Analysis of the company's liquidity.
    """
    # Define the prompt to analyze liquidity based on the provided data
    prompt = """
    Analyze the company's liquidity based on the following data:
    - Total Assets: ${total_assets} million
    - Total Liabilities: ${total_liabilities} million

    Calculate the current ratio and provide a brief analysis of the company's liquidity.
    """
    # Call the function to get the model's response for liquidity analysis
    return run_prompt(prompt, total_assets=total_assets, total_liabilities=total_liabilities)

# Function to analyze the company's cash flow
def analyze_cash_flow(cash_flow):
    """Analyze the company's cash flow.

    Args:
        cash_flow (float): Company's cash flow from operations.

    Returns:
        str: Analysis of the company's cash flow.
    """
    # Define the prompt to analyze cash flow based on the provided data
    prompt = """
    Analyze the company's cash flow based on the following data:
    - Cash Flow from Operations: ${cash_flow} million

    Provide a brief analysis of the company's cash flow health.
    """
    # Call the function to get the model's response for cash flow analysis
    return run_prompt(prompt, cash_flow=cash_flow)

# Run the chained subtasks
profitability_analysis = analyze_profitability(10, 2)
liquidity_analysis = analyze_liquidity(15, 7)
cash_flow_analysis = analyze_cash_flow(3)

# Print the results of all three analyses
print("Profitability Analysis:\n", profitability_analysis)
print("\nLiquidity Analysis:\n", liquidity_analysis)
print("\nCash Flow Analysis:\n", cash_flow_analysis)

Profitability Analysis:
 To analyze the company's profitability, we will calculate the profit margin using the provided data.

### Profit Margin Calculation

The profit margin is calculated using the formula:

\[
\text{Profit Margin} = \left( \frac{\text{Net Income}}{\text{Revenue}} \right) \times 100
\]

Substituting the given values:

- **Net Income**: $2 million
- **Revenue**: $10 million

\[
\text{Profit Margin} = \left( \frac{2 \, \text{million}}{10 \, \text{million}} \right) \times 100 = 20\%
\]

### Analysis of Profitability

A profit margin of **20%** indicates that the company retains 20 cents of profit for every dollar of revenue generated. This is a strong profit margin, suggesting that the company is effectively managing its costs and expenses relative to its sales.

### Key Points:
1. **Efficiency**: A 20% profit margin typically signifies good operational efficiency. The company is likely controlling its costs well, which is essential for long-term profitability.
2. **Com

1. `analyze_profitability(revenue, net_income)`: This function takes in two parameters: `revenue` and `net_income`. The prompt provided to the language model requests it to analyze the company's profitability based on these two data points. The model is asked to calculate the profit margin (which is generally calculated as `Net Income / Revenue`) and provide a brief analysis of the company’s profitability.
2. `analyze_liquidity(total_assets, total_liabilities)`: This function works similarly to the previous one but focuses on liquidity. The parameters `total_assets` and `total_liabilities` are used to evaluate the company's liquidity by calculating its current ratio (which is typically `Total Assets / Total Liabilities`).
3. `analyze_cash_flow(cash_flow)`: In this function, the focus shifts to analyzing the company’s cash flow from operations. The model is prompted to provide an analysis of the company’s cash flow health, which is a key indicator of the company’s operational efficiency and its ability to generate cash.


### Integrating results
Once we have the individual analyses from the subtasks—profitability, liquidity, and cash flow—we need to combine them into a single, comprehensive assessment of the company's overall financial health. This step is crucial because, while each analysis provides valuable insights into a specific financial aspect of the company, it is only by considering them together that we can form a full picture of the company's financial position.

In this section, we will demonstrate how to integrate the results from the subtasks and synthesize them into an overall financial health assessment. This is achieved by creating a new prompt that includes the outputs from each subtask as input, and instructing the language model to summarize and evaluate the company's financial situation based on all the provided information.

In [6]:
# Function to integrate results from subtasks to provide an overall analysis
def integrate_results(profitability, liquidity, cash_flow):
    """Integrate the results from subtasks to provide an overall analysis.

    Args:
        profitability (str): Profitability analysis.
        liquidity (str): Liquidity analysis.
        cash_flow (str): Cash flow analysis.

    Returns:
        str: Overall analysis of the company's financial health.
    """
    # Define the prompt for integrating results into an overall analysis
    prompt = """
    Based on the following analyses, provide an overall assessment of the company's financial health:

    Profitability Analysis:
    {profitability}

    Liquidity Analysis:
    {liquidity}

    Cash Flow Analysis:
    {cash_flow}

    Summarize the key points and give an overall evaluation of the company's financial position.
    """
    # Pass the results from each analysis into the prompt and invoke the model to generate the overall analysis
    return run_prompt(prompt, profitability=profitability, liquidity=liquidity, cash_flow=cash_flow)

# Integrate the results of the three analyses into an overall financial health assessment
overall_analysis = integrate_results(profitability_analysis, liquidity_analysis, cash_flow_analysis)
print("Overall Financial Health Analysis:\n", overall_analysis)

Overall Financial Health Analysis:
 ### Overall Assessment of the Company's Financial Health

The company's financial health can be evaluated through three key analyses: profitability, liquidity, and cash flow. Here's a summary of the findings from each analysis:

#### Profitability Analysis
- **Profit Margin**: The company has a profit margin of **20%**, meaning it retains 20 cents of profit for every dollar of revenue. 
  - **Efficiency**: This indicates strong operational efficiency and effective cost management.
  - **Comparison**: A comparison to industry averages could further validate its competitive positioning.
  - **Sustainability**: While the current margin is promising, ongoing assessment will be necessary to ensure it remains sustainable amidst market fluctuations.

#### Liquidity Analysis
- **Total Assets vs. Total Liabilities**: The company has total assets of **$15 million** and total liabilities of **$7 million**, providing a significant buffer to cover its obligations

1. `integrate_results(profitability, liquidity, cash_flow)`:This function is responsible for integrating the results from the three separate analyses into an overall financial assessment. The function creates a prompt that presents all three analyses to the language model and asks it to synthesize the key points from each one and provide an overall evaluation.
2. Prompt design: The prompt is designed to be comprehensive and detailed. It includes placeholders for the results of the three analyses (profitability, liquidity, and cash flow). The model is asked to summarize the key points from each analysis and then provide an overarching evaluation of the company’s financial health. This prompt allows the model to process and integrate multiple aspects of the company's financial situation, leading to a cohesive and informative output.


Breaking the task into subtasks and then integrating the results ensures that the process is systematic and efficient. Each analysis focuses on a specific area, while the integration step combines them into one comprehensive assessment.

This approach makes the problem-solving process more manageable and ensures that the final output is well-rounded and aligned with the overall goal: to provide a comprehensive evaluation of the company’s financial health. This method ensures that each individual analysis is thorough, while the final integrated output gives a holistic view of the company's financial situation.